<center> <img src="images/HSSM_logo.png"> </center>

This tutorial provides a comprehensive introduction to the HSSM package for Hierarchical Bayesian Estimation of Sequential Sampling Models.

To make the most of the tutorial, let us cover the functionality of the key supporting packages that we use along the way. 

## Colab Instructions

If you would like to run this tutorial on Google colab, please click this [link](https://github.com/lnccbrown/HSSM/blob/main/docs/tutorials/main_tutorial.ipynb). 

Once you are *in the colab*, follow the *installation instructions below* and then **restart your runtime**. 

Just **uncomment the code in the next code cell** and run it!

**NOTE**:

You may want to *switch your runtime* to have a GPU or TPU. To do so, go to *Runtime* > *Change runtime type* and select the desired hardware accelerator.

Note that if you switch your runtime you have to follow the installation instructions again.

In [1]:
# If running this on Colab, please uncomment the next line
# !pip install git+https://github.com/lnccbrown/HSSM@workshop_tutorial

## Basic Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

# Basics
import os
import sys
import time

import numpy as np
from matplotlib import pyplot as plt

random_seed_sim = 134
np.random.seed(random_seed_sim)

## Data Simulation

We will rely on the [ssms](https://github.com/AlexanderFengler/ssm-simulators) package for data simulation repeatedly. Let's look at a basic isolated use case below.

As an example, let's use [ssms](https://github.com/AlexanderFengler/ssm-simulators) to simulate from the basic [Drift Diffusion Model](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2474742/) (a running example in this tutorial).

<center> <img src="images/DDM_with_params_pic.png" height="400" width="400"> </center>

If you are not familiar with the DDM. For now just consider that it has four parameters. 

- `v` the drift rate
- `a` the boundary separation
- `t` the non-decision time
- `z` the a priori decision bias (starting point)

### Using `simulate_data()`

HSSM comes with a basic simulator function supplied the `simulate_data()` function. We can use this function to create synthetic datasets.

Below we show the most basic usecase: 

We wish to generate `500` datapoints (trials) from the standard [Drift Diffusion Model](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2474742/) with a fixed parameters, `v = 0.5, a = 1.5, z = 0.5, t = 0.5`. 


**Note**:

In the course of the tutorial, we will see multiple strategies for synthetic dataset generation, this being the most straightforward one. 


In [7]:
# Single dataset
import hddm_wfpt
import hssm
import pytensor  # Graph-based tensor library
import jax
import bambi as bmb # Model construction 
import arviz as az  # Visualization

pytensor.config.floatX = "float32"
jax.config.update("jax_enable_x64", False)

v_true = 0.5
a_true = 1.5
z_true = 0.5
t_true = 0.5

# Call the simulator function
dataset = hssm.simulate_data(model = 'ddm',
							 theta = dict(v = v_true,
										  a = a_true,
										  z = z_true,
										  t = t_true),
							 size = 500)

dataset

,rt,response
0,3.791911,-1.0
1,1.094867,-1.0
2,5.845557,1.0
3,0.989894,1.0
4,2.683095,1.0
...,...,...
495,2.167489,1.0
496,1.480039,1.0
497,3.552173,-1.0
498,2.129561,1.0


If instead you wish to supply a parameter that *varies by trial* (a lot more on this later), you can simply supply a vector of parameters to the `theta` dictionary, when calling the simulator.

**Note**:

The `size` argument conceptually functions as *number of synthetic datasets*. So if you supply a parameter as a `(1000,)` vector, then the simulator assumes that one dataset consists of `1000` trials, hence if we set the `size = 1` as below, we expect in return a dataset with `1000` trials. 

In [4]:
# a changes trial wise
a_trialwise = np.random.normal(loc=2, scale=0.3, size=1000)

dataset_a_trialwise = hssm.simulate_data(model = 'ddm',
                                         theta = dict(v = v_true,
                                                      a = a_trialwise,
                                                      z = z_true, 
                                                      t = t_true,
                                                      ),          
                                         size = 1)

dataset_a_trialwise

,rt,response
0,3.541487,1.0
1,1.877541,1.0
2,1.633046,1.0
3,2.677460,1.0
4,3.091263,1.0
...,...,...
995,3.208477,1.0
996,2.262080,1.0
997,1.910646,1.0
998,2.647852,1.0


If we wish to simulate from another model, we can do so by changing the `model` string.

The number of models we can simulate differs from the number of models for which we have likelihoods available (both will increase over time). To get the models for which likelihood functions are supplied out of the box, we should check the `SupportedModels` under `hssm.defaults`.

In [5]:
hssm.defaults.SupportedModels

typing.Literal['ddm', 'ddm_sdv', 'full_ddm', 'angle', 'levy', 'ornstein', 'weibull', 'race_no_bias_angle_4', 'ddm_seq2_no_bias']

If we wish to check more detailed information about a given model, we can use the `default_model_config` under `hssm.default`.

Let's look at the `ddm`:

In [6]:
hssm.defaults.default_model_config['angle']

{'response': ['rt', 'response'],
 'list_params': ['v', 'a', 'z', 't', 'theta'],
 'description': None,
 'likelihoods': {'approx_differentiable': {'loglik': 'angle.onnx',
   'backend': 'jax',
   'default_priors': {},
   'bounds': {'v': (-3.0, 3.0),
    'a': (0.3, 3.0),
    'z': (0.1, 0.9),
    't': (0.001, 2.0),
    'theta': (-0.1, 1.3)},
   'extra_fields': None}}}

This dictionary contains quite a bit of information. For purposes of *simulating data from a given model*, we will highlight two aspects:

1.  The key `list_of_params` provides us with the necessary information to define out `theta` dictionary
2.  The `bounds` key inside the `likelihoods` sub-dictionaries, provides us with an indication of reasonable parameter values.

The `likelihoods` dictionary inhabits three sub-directories for the `ddm` model, since we have all three, an `analytical`, an `approx_differentiable` (LAN) and a `blackbox` likelihood available. For many models, we will be able to access only one or two types of likelihoods.

### Using `ssm-simulators`

Internally, HSSM natively makes use of the [ssm-simulators](https://github.com/AlexanderFengler/ssm-simulators) package for forward simulation of models. 
`hssm.simulate_data()` functions essentially as a convenience-wrapper. 

Below we illustrate how to simulate data using the `ssm-simulators` package directly, to generate an equivalent dataset as created above. We will use the *third* way of passing parameters to the simulator, which is as a parameter-*matrix*. 

**Notes**:

1. If you pass parameters as a parameter matrix, make sure the column ordering is correct. You can follow the parameter ordering under `hssm.defaults.default_model_config['ddm']['list_params']`.

2. This is a minimal example, for more information about the package, check the associated [github-page](https://github.com/AlexanderFengler/ssm-simulators). 





In [8]:
import ssms
import pandas as pd
from ssms.basic_simulators.simulator import simulator

# a changes trial wise
theta_mat = np.zeros((1000, 4))
theta_mat[:, 0] = v_true # v
theta_mat[:, 1] = a_trialwise # a
theta_mat[:, 2] = z_true # z
theta_mat[:, 3] = t_true # t

# simulate data
sim_out_trialwise = simulator(
    theta=theta_mat,  # parameter_matrix
    model="ddm",  # specify model (many are included in ssms)
    n_samples=1,  # number of samples for each set of parameters (plays the role of `size` parameter in `hssm.simulate_data`)
)

# Turn into nice dataset
dataset_trialwise = pd.DataFrame(
    np.column_stack(
        [sim_out_trialwise["rts"][:, 0], sim_out_trialwise["choices"][:, 0]]
    ),
    columns=["rt", "response"],
)

dataset_trialwise

NameError: name 'np' is not defined

We will stick to `hssm.simulate_data()` in this tutorial, to keep things simple.

## ArviZ for Plotting

<center> <img src="images/arviz.png" height="200" width="200"> </center>


We use the [ArviZ](https://python.arviz.org/en/stable/) package for most of our plotting needs. 
ArviZ is a useful aid for plotting when doing anything Bayesian. 

It works with HSSM out of the box, by virtue of HSSMs reliance on [PyMC](https://www.pymc.io/welcome.html) for model construction and sampling.

Checking out the [ArviZ Documentation](https://python.arviz.org/en/stable/getting_started/index.html) is a good idea to give you communication superpowers for not only your HSSM results, but also other libraries in the Bayesian Toolkit such as [NumPyro](https://num.pyro.ai/en/latest/index.html#introductory-tutorials) or [STAN](https://mc-stan.org/users/documentation/).

We will see [ArviZ](https://python.arviz.org/en/stable/) plots throughout the notebook.  

# Main Tutorial

## Initial Dataset

Let's proceed to simulate a simple dataset for our first example.

In [9]:
# Specify 
param_dict_init = dict(v = v_true,
                       a = a_true,
                       z = z_true,
                       t = t_true,
                      )


dataset = hssm.simulate_data(model = 'ddm',
                   theta = param_dict_init,          
                   size = 500,
                   )

dataset

,rt,response
0,1.873666,1.0
1,3.317008,1.0
2,2.017146,1.0
3,2.252344,-1.0
4,1.053887,1.0
...,...,...
495,1.585201,1.0
496,3.210716,1.0
497,6.281962,-1.0
498,1.205276,1.0


## First HSSM Model

In this example we will use the *analytical likelihood function* computed as suggested in [this paper](https://psycnet.apa.org/record/2009-11068-003).

### Instantiate the model

To instantiate our `HSSM` class, in the simplest version, we only need to provide an appropriate dataset.
The dataset is expected to be a `pandas.DataFrame` with at least two columns, respectively called `rt` (for reaction time) and `response`.
Our data simulated above is already in the correct format, so let us try to construct the class.

**NOTE:**

If you are a user of the [HDDM](https://github.com/hddm-devs/hddm) python package, this workflow should seem very familiar.

In [10]:
simple_ddm_model = hssm.HSSM(data=dataset)

In [11]:
simple_ddm_model

Hierarchical Sequential Sampling Model
Model: ddm

Response variable: rt,response
Likelihood: analytical
Observations: 500

Parameters:

v:
    Prior: Normal(mu: 0.0, sigma: 2.0)
    Explicit bounds: (-inf, inf)

a:
    Prior: HalfNormal(sigma: 2.0)
    Explicit bounds: (0.0, inf)

z:
    Prior: Uniform(lower: 0.0, upper: 1.0)
    Explicit bounds: (0.0, 1.0)

t:
    Prior: HalfNormal(sigma: 2.0)
    Explicit bounds: (0.0, inf)


Lapse probability: 0.05
Lapse distribution: Uniform(lower: 0.0, upper: 10.0)

The `print()` function gives us some basic information about our model including the *number of observations* the *parameters in the model* and their respective *prior setting*. We can also create a nice little graphical representation of our model...

### Model Graph

Since `HSSM` creates a `PyMC Model`, we can can use the `.graph()` function, to get a graphical representation of the the model we created.

In [16]:
from hssm.utils import HSSMModelGraph

In [18]:
ModelGraph = HSSMModelGraph(model=simple_ddm_model.pymc_model, parent=simple_ddm_model._parent_param)

TypeError: cannot unpack non-iterable Plate object

In [19]:
ModelGraph.make_graph(formatting='plain', response_str=simple_ddm_model.response_str)

AttributeError: 'list' object has no attribute 'items'

In [12]:
simple_ddm_model.graph()

AttributeError: 'list' object has no attribute 'items'

This is the simplest model we can build. The graph above follows **plate notation**, commonly used for **probabilistic graphical models**.

- We have our basic parameters (unobserved, **white nodes**), these are *random variables* in the model and we want to estimate them
- Our observed reaction times and choices (`SSMRandomVariable`, **grey node**), are fixed (or conditioned on).
- **Rounded rectangles** provide us with information about dimensionality of objects
- **Rectangles with sharp edges** represent *deterministic*, but *computed* quantities (not shown here, but in later models)

This notation is helpful to get a quick overview of the structure of a given model we construct.

The `graph()` function of course becomes a lot more interesting and useful for more complicated models!

### Sample from the Model

We can now call the `.sample()` function, to get posterior samples. The main arguments you may want to change are listed in the function call below. 

Importantly, multiple backends are possible. We choose the `nuts_numpyro` backend below,
which in turn compiles the model to a [`JAX`](https://github.com/google/jax) function.

In [ ]:
infer_data_simple_ddm_model = simple_ddm_model.sample(
    sampler="nuts_numpyro",  # type of sampler to choose, 'nuts_numpyro', 'nuts_blackjax' of default pymc nuts sampler
    cores=1,  # how many cores to use
    chains=2,  # how many chains to run
    draws=500,  # number of draws from the markov chain
    tune=1000,  # number of burn-in samples
    idata_kwargs=dict(log_likelihood=True),  # return log likelihood
)  # mp_ctx="forkserver")

We sampled from the model, let's look at the output...

In [ ]:
type(infer_data_simple_ddm_model)

Errr... a closer look might be needed here!

### Inference Data / What gets returned from the sampler?

The sampler returns an [ArviZ](https://python.arviz.org/en/stable/) `InferenceData` object. 

To understand all the logic behind these objects and how they mesh with the Bayesian Workflow, we refer you to the [ArviZ Documentation](https://python.arviz.org/en/stable/getting_started/index.html). 

`InferenceData` is build on top of [xarrays](https://docs.xarray.dev/en/stable/index.html). The [xarray documentation](https://docs.xarray.dev/en/stable/index.html) will help you understand in more detail how to manipulate these objects.

But let's take a quick high-level look to understand roughly what we are dealing with here!

In [ ]:
infer_data_simple_ddm_model

We see that in our case, `infer_data_simple_ddm_model` contains four basic types of data (note: this is extensible!)

- `posterior`
- `log_likelihood`
- `sample_stats`
- `observed_data`

The `posterior` object contains our traces for each of the parameters in the model. The `log_likelihood` field contains the trial wise log-likelihoods for each sample from the posterior. The `sample_stats` field contains information about the sampler run. This can be important for chain diagnostics, but we will not dwell on this here. Finally we retreive our `observed_data`.


### Basic Manipulation

#### Accessing groups and variables

In [ ]:
infer_data_simple_ddm_model.posterior

In [ ]:
infer_data_simple_ddm_model.posterior.a.head()

To simply access the underlying data as a `numpy.ndarray`, we can use `.values` (as e.g. when using `pandas.DataFrame` objects). 

In [ ]:
type(infer_data_simple_ddm_model.posterior.a.values)

In [ ]:
#infer_data_simple_ddm_model.posterior.a.values

#### Combine `chain` and `draw` dimension

When operating directly on the `xarray`, you will often find it useful to collapse the `chain` and `draw` coordinates into a single coordinate.
**Arviz** makes this easy via the `extract` method.

In [ ]:
idata_extracted = az.extract(infer_data_simple_ddm_model)
idata_extracted

Since *Arviz* really just calls the `.stack()` method from *xarray*, here the corresponding example using the lower level `xarray` interface.

In [ ]:
infer_data_simple_ddm_model.posterior.stack(sample=('chain', 'draw'))

### Making use of ArviZ

Working with the `InferenceData` directly, is very helpful if you want to include custom computations into your workflow. 
For a basic Bayesian Workflow however, you will often find that standard functionality available through [ArviZ](https://python.arviz.org/en/stable/)
suffices.

Below we provide a few examples of useful **Arviz** outputs, which come handy for analyzing your traces (MCMC samples).

#### Summary table

Let's take a look at a summary table for our posterior. 

In [ ]:
az.summary(infer_data_simple_ddm_model, 
           var_names = [var_name.name for var_name in simple_ddm_model.pymc_model.free_RVs])

This table returns the parameter-wise mean of our posterior and a few extra statistics.

Of these extra statistics, the one-stop shop for flagging convergence issues is the `r_hat` value, which 
is reported in the right-most column. 

To navigate this statistic, here is a rule of thumb widely used in applied Bayesian statistics.

If you find an `r_hat` value `> 1.01`, it warrants investigation.

#### Trace plot

In [ ]:
az.plot_trace(
    infer_data_simple_ddm_model,  # we exclude the log_likelihood traces here
    lines = [(key_, {}, param_dict_init[key_]) for key_ in param_dict_init],
)
plt.tight_layout()

The `.sample()` function also sets a `trace` attribute, on our `hssm` class, so instead, we could call the plot like so:

In [ ]:
az.plot_trace(simple_ddm_model.traces,
              lines = [(key_, {}, param_dict_init[key_]) for key_ in param_dict_init],
             );

In this tutorial we are most often going to use the latter way of accessing the traces, but there is no preferred option. 

Let's look at a few more plots.

#### Forest Plot

The forest plot is commonly used for a quick visual check of the marginal posteriors. It is very effective for intuitive communication of results.

In [ ]:
az.plot_forest(simple_ddm_model.traces)

##### Combining Chains
By default, chains are separated out into *separate caterpillars*, however
sometimes, especially if you are looking at a forest plot which includes many posterior parameters at once, you want to declutter and collapse the chains into single caterpillars.
In this case you can `combine` chains instead.

In [ ]:
az.plot_forest(simple_ddm_model.traces, combined=True)

#### Basic Marginal Posterior Plot

Another way to view the marginal posteriors is provided by the `plot_posterior()` function. It shows the mean and by default the $94\%$ HDIs.

In [ ]:
az.plot_posterior(simple_ddm_model.traces)

Especially for parameter recovery studies, you may want to include **reference values** for the parameters of interest. 

You can do so using the `ref_val` argument. See the example below:

In [ ]:
az.plot_posterior(simple_ddm_model.traces, 
                  ref_val = [param_dict_init[var_name] for var_name in simple_ddm_model.traces.posterior.data_vars])

Since it is sometimes useful, especially for more complex cases, below an alternative approach in which we pass `ref_val` as a dictionary.


In [ ]:
az.plot_posterior(simple_ddm_model.traces, ref_val = {'v': [{'ref_val': param_dict_init['v']}],
                                                      'a': [{'ref_val': param_dict_init['a']}],
                                                      'z': [{'ref_val': param_dict_init['z']}],
                                                      't': [{'ref_val': param_dict_init['t']}]})

#### Posterior Pair Plot

The posterior pair plot show us bi-variate traceplots and is useful to check for simple parameter tradeoffs that may emerge. The simplest (linear) tradeoff may be a high correlation between two parameters.
This can be very helpful in diagnosing sampler issues for example. If such tradeoffs exist, one often see extremely *wide marginal distributions*.

In our `ddm` example, we see a little bit of a tradeoff between `a` and `t`, as well as between `v` and `z`, however nothing concerning. 

In [ ]:
az.plot_pair(simple_ddm_model.traces, 
             kind="kde", 
             reference_values = param_dict_init, 
             marginals = True);

The few plot we showed here are just the beginning: [ArviZ](https://python.arviz.org/en/stable/) has a much broader spectrum of graphs and other convenience function available. Just check the [documentation](https://python.arviz.org/en/stable/). 

### Compute Quantities from idata

#### Example: Mean and Covariance of Posterior Parameters

As a simple example, let us calculate the covariance matrix for our posterior samples.

In [ ]:
# Calculate the correlation matrix
posterior_correlation_matrix = np.corrcoef(np.stack([idata_extracted[var_].values for var_ in idata_extracted.data_vars.variables]))
num_vars = posterior_correlation_matrix.shape[0]

# Make heatmap
fig, ax = plt.subplots(1,1)
cax = ax.imshow(posterior_correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax, ax=ax)
ax.set_title("Posterior Correlation Matrix")

# Add ticks
ax.set_xticks(range(posterior_correlation_matrix.shape[0]))
ax.set_xticklabels([var_ for var_ in idata_extracted.data_vars.variables])
ax.set_yticks(range(posterior_correlation_matrix.shape[0]))
ax.set_yticklabels([var_ for var_ in idata_extracted.data_vars.variables])

# Annotate heatmap
for i in range(num_vars):
    for j in range(num_vars):
        ax.text(j, i, f'{posterior_correlation_matrix[i, j]:.2f}', ha='center', va='center', color='black')

plt.show()

## HSSM Model based on LAN likelihood

With HSSM you can switch between pre-supplied models with a simple change of argument. The type of likelihood that will be accessed might change in the background for you. 

Here we see an example in which the underlying likelihood is now a [LAN](https://elifesciences.org/articles/65074).

We will talk more about different types of likelihood functions and backends later in the tutorial. For now just keep the following in mind:

There are three types of likelihoods

1. `analytic`
2. `approx_differentiable`
3. `blackbox`

To check which type is used in your HSSM model simple type:

In [ ]:
simple_ddm_model.loglik_kind

Ah... we were using an `analytical` likelihood with the DDM model in the last section.
Now let's see something different!

### Simulating Angle Data

Again, let us simulate a simple dataset. This time we will use the `angle` model (passed via the `model` argument to the `simulator()` function). 


This model is distinguished from the basic `ddm` model by an additional `theta` parameter which specifies the angle with which the decision boundaries collapse over time.

<center> <img src="images/ANGLE_with_params_pic.png" height="400" width="400"> </center>

DDMs with collapsing bounds have been of significant interest in the theoretical literature, but applications were rare due to a lack of analytical likelihoods. HSSM facilitates inference with such models via the our `approx_differentiable` likelihoods. HSSM ships with a few predefined models based on [LANs](https://elifesciences.org/articles/65074), but really we don't want to overemphasize those. They reflect the research interest of our and adjacent labs to a great extend. 

Instead, we encourage the community to contribute to this model reservoir (more on this later). 

In [ ]:
# Simulate angle data
v_angle_true = 0.5
a_angle_true = 1.5
z_angle_true = 0.5
t_angle_true = 0.2
theta_angle_true = 0.2

param_dict_angle = dict(v = 0.5, 
                        a = 1.5, 
                        z = 0.5,
                        t = 0.2,
                        theta = 0.2)

lines_list_angle = [(key_, {}, param_dict_angle[key_]) for key_ in param_dict_angle]

dataset_angle = hssm.simulate_data(model = 'angle',
                                   theta = param_dict_angle,
                                   size = 1000)

We pass a single additional argument to our `HSSM` class and set `model='angle'`. 

In [ ]:
model_angle = hssm.HSSM(data=dataset_angle, model="angle")

model_angle

The model graph now show us an additional parameter `theta`!

In [ ]:
model_angle.graph()

Let's check the type of likelihood that is used under the hood ...

In [ ]:
model_angle.loglik_kind

Ok so here we rely on a likelihood of the `approx_differentiable` kind. 

As discussed, with the initial set of pre-supplied likelihoods, this implies that we are using a [LAN](https://elifesciences.org/articles/65074) in the background.

In [ ]:
jax.config.update("jax_enable_x64", False)
infer_data_angle = model_angle.sample(
    sampler="nuts_numpyro",
    chains=2,
    cores=2,
    draws=500,
    tune=500,
    idata_kwargs=dict(log_likelihood=False),  # no need to return likelihoods here
)

In [ ]:
az.plot_trace(model_angle.traces, lines = lines_list_angle)
plt.tight_layout()

## Choosing Priors

HSSM allows you to specify priors quite freely. If you used HDDM previously, you may feel relieved to read that your hands are now untied!


<center><img src="https://media.giphy.com/media/99uUPYHtmaJS8/giphy.gif" style="margin:auto" height="400" width="400"/></center>

With HSSM we have multiple routes to priors. But let's first consider a special case:

### Fixing a parameter to a given value

Assume that instead of fitting all parameters of the DDM, 

<center> <img src="images/DDM_with_params_pic.png" height="400" width="400"> </center> 

we instead want to fit only the `v` (drift) parameter, setting all other parameters to fixed scalar values.

<center> <img src="images/DDM_only_v_pic.png" height="400" width="400"> </center> 

HSSM makes this extremely easy!

In [ ]:
param_dict_init

In [ ]:
ddm_model_only_v = hssm.HSSM(data=dataset, model="ddm", 
                             a = param_dict_init['a'], 
                             t = param_dict_init['t'], 
                             z = param_dict_init['z'])

Since we fix all but one parameter, we therefore estimate only one parameter. This should be reflected in our model graph, where we expect only one free random variable `v`:

In [ ]:
ddm_model_only_v.graph()

In [ ]:
ddm_model_only_v.sample(
    sampler="nuts_numpyro",
    chains=2,
    cores=2,
    draws=500,
    tune=500,
    idata_kwargs=dict(log_likelihood=False),  # no need to return likelihoods here
)

In [ ]:
az.plot_trace(ddm_model_only_v.traces.posterior, lines = [('v', {}, param_dict_init['v'])]);

Instead of the trace on the right, a useful alternative / complement is the **rank plot**.
As a rule of thumb, if the rank plots within chains look *uniformly distributed*, then our chains generally exhibit *good mixing*.

In [ ]:
az.plot_trace(ddm_model_only_v.traces, kind = 'rank_bars')

### Named priors

We can choose any [PyMC](https://www.pymc.io/welcome.html) `Distribution` to specify a prior for a given parameter. 

Even better, if natural *parameter bounds* are provided, HSSM *automatically truncates the prior distribution* so that it respect these bounds. 

Below is an example in which we specify a *Normal* prior on the `v` parameter of the DDM. 

We choose a *ridiculously low* $\sigma$ value, to illustrate it's regularizing effect on the parameter (just so we see a difference and you are convinced that something changed). 

In [ ]:
model_normal = hssm.HSSM(
    data=dataset,
    include=[
        {
            "name": "v",
            "prior": {"name": "Normal", "mu": 0, "sigma": 0.01},
        }
    ],
)

In [ ]:
model_normal

In [ ]:
infer_data_normal = model_normal.sample(
    sampler="nuts_numpyro",
    chains=2,
    cores=2,
    draws=500,
    tune=500,
    idata_kwargs=dict(log_likelihood=False),  # no need to return likelihoods here
)

In [ ]:
az.plot_trace(model_normal.traces, 
              lines = [(key_, {}, param_dict_init[key_]) for key_ in param_dict_init])

Observe how we reused our previous dataset with underlying parameters 

- `v = 0.5`
- `a = 1.5`
- `z = 0.5`
- `t = 0.2`

In contrast to our previous sampler round, in which we used Uniform priors, here the `v` estimate is shrunk severley towared $0$ and the `t` and `z` parameter estimates are very biased to make up for this distortion.

## HSSM Model with Regression


<center> <img src="images/bambi.png" height="200" width="200"> </center>

Crucial to the scope of HSSM is the ability to link parameters with trial-by-trial covariates via (hierarchical, but more on this later) general linear models. 
 
In this section we explore how HSSM deals with these models. No big surprise here... it's simple!

### Case 1: One parameter is a Regression Target 

#### Simulating Data

Let's first simulate some data, where the trial-by-trial parameters of the `v` parameter in our model are driven by a simple linear regression model.


The regression model is driven by two (random) covariates `x` and `y`, respectively with coefficients of $0.8$ and $0.3$ which are also simulated below.
We set the intercept to $0.3$.

The rest of the parameters are fixed to single values as before.

In [ ]:
# Set up trial by trial parameters
v_intercept = 0.3
x = np.random.uniform(-1, 1, size=1000)
v_x = 0.8
y = np.random.uniform(-1, 1, size=1000)
v_y = 0.3
v_reg_v = v_intercept + (v_x * x) + (v_y * y)

# rest
a_reg_v = 1.5
z_reg_v = 0.5
t_reg_v = 0.1

param_dict_reg_v = dict(a = 1.5,
                        z = 0.5,
                        t = 0.1,
                        v = v_reg_v,
                        v_x = v_x,
                        v_y = v_y,
                        v_Intercept = v_intercept,
                        theta = 0.0)

# base dataset
dataset_reg_v = hssm.simulate_data(model = 'ddm',
                                   theta = param_dict_reg_v,
                                   size = 1)

# Adding covariates into the datsaframe
dataset_reg_v['x'] = x
dataset_reg_v['y'] = y

#### Basic Model

We now create the `HSSM` model. 

Notice how we set the `include` argument. The include argument expects a list of dictionaries, one dictionary for each parameter to be specified via a regression model. 

Four `keys` are expected to be set:

- The `name` of the parameter, 
- Potentially a `prior` for each of the regression level parameters ($\beta$'s), 
- The regression `formula` 
- A `link` function. 

The regression formula follows the syntax in the [formulae](https://pypi.org/project/formulae/) python package (as used by the [Bambi](https://bambinos.github.io/bambi/) package for building Bayesian Hierarchical Regression Models. 

[Bambi](https://bambinos.github.io/bambi/) forms the main model-construction backend of HSSM.

In [ ]:
model_reg_v_simple = hssm.HSSM(
    data = dataset_reg_v,
    include = [{"name": "v",
                "formula": "v ~ 1 + x + y"}])

In [ ]:
model_reg_v_simple

##### `Param` class
As illustrated below, there is an alternative way of specifying the parameter specific data via the `Param` class.

In [ ]:
model_reg_v_simple_new = hssm.HSSM(
    data = dataset_reg_v,
    include = [hssm.Param(name = 'v', formula='v ~ 1 + x + y')])

In [ ]:
model_reg_v_simple_new

In [ ]:
model_reg_v_simple.graph()

In [ ]:
print(model_reg_v_simple)

#### Custom Model

These were the defaults, with a little extra labor, we can e.g. customize the choice of priors for each parameter in the model.

In [ ]:
model_reg_v = hssm.HSSM(
    data=dataset_reg_v,
    include=[
        {
            "name": "v",
            "prior": {
                "Intercept": {"name": "Uniform", "lower": -3.0, "upper": 3.0},
                "x": {"name": "Uniform", "lower": -1.0, "upper": 1.0},
                "y": {"name": "Uniform", "lower": -1.0, "upper": 1.0},
            },
            "formula": "v ~ 1 + x + y",
            "link": "identity",
        }
    ],
)

In [ ]:
model_reg_v

Notice how `v` is now set as a regression.

In [ ]:
infer_data_reg_v = model_reg_v.sample(
    sampler="nuts_numpyro", chains=1, cores=1, draws=500, tune=500
)

In [ ]:
infer_data_reg_v

In [ ]:
#az.plot_forest(model_reg_v.traces)

In [ ]:
az.plot_trace(model_reg_v.traces, 
              var_names = ['~v'],
              lines = [(key_, {}, param_dict_reg_v[key_]) for key_ in param_dict_reg_v])

In [ ]:
az.plot_trace(model_reg_v.traces, 
              var_names = ['~v'],
              lines = [(key_, {}, param_dict_reg_v[key_]) for key_ in param_dict_reg_v])
plt.tight_layout()

In [ ]:
# Looks like parameter recovery was successful
az.summary(model_reg_v.traces, var_names = ['~v'])

### Case 2: One parameter is a Regression (LAN)

We can do the same thing with the `angle` model.

**Note**: 

Our dataset was generated from the basic DDM here, so since the DDM assumes stable bounds, we expect the `theta` (angle of linear collapse) parameter to be recovered as close to $0$.

In [ ]:
model_reg_v_angle = hssm.HSSM(
    data=dataset_reg_v,
    model="angle",
    include=[
        {
            "name": "v",
            "prior": {
                "Intercept": {
                    "name": "Uniform",
                    "lower": -3.0,
                    "upper": 3.0,
                    # "initval": 0   # optional --> set the initial value of the parameter (to e.g. avoid boundary violations at the intial sampling step)
                },
                "x": {
                    "name": "Uniform",
                    "lower": -1.0,
                    "upper": 1.0,
                },
                "y": {"name": "Uniform", "lower": -1.0, "upper": 1.0},
            },
            "formula": "v ~ 1 + x + y",
            "link": "identity",
        }
    ],
)

In [ ]:
model_reg_v_angle.graph()

In [ ]:
trace_reg_v_angle = model_reg_v_angle.sample(
    sampler="nuts_numpyro", chains=1, cores=1, draws=1000, tune=500
)

In [ ]:
az.plot_trace(model_reg_v_angle.traces, 
              var_names = ['~v'],
              lines = [(key_, {}, param_dict_reg_v[key_]) for key_ in param_dict_reg_v])
plt.tight_layout()

Great! `theta` is recovered correctly, on top of that, we have reasonable recovery for all other parameters!

### Case 3: Multiple Parameters are Regression Targets (LAN) 

Let's get a bit more ambitious. We may, for example, want to try a regression on a few of our basic model parameters at once. Below we show an example where we model both the `a` and the `v` parameters with a regression.

**NOTE:**

In our dataset of this section, only `v` is *actually* driven by a trial-by-trial regression, so we expect the regression coefficients for `a` to hover around $0$ in our posterior.

In [ ]:
# Instantiate our hssm model
from copy import deepcopy
param_dict_reg_v_a = deepcopy(param_dict_reg_v)
param_dict_reg_v_a['a_Intercept'] = param_dict_reg_v_a['a']
param_dict_reg_v_a['a_x'] = 0
param_dict_reg_v_a['a_y'] = 0

hssm_reg_v_a_angle = hssm.HSSM(
    data=dataset_reg_v,
    model="angle",
    include=[
        {
            "name": "v",
            "prior": {
                "Intercept": {"name": "Uniform", "lower": -3.0, "upper": 3.0},
                "x": {"name": "Uniform", "lower": -1.0, "upper": 1.0},
                "y": {"name": "Uniform", "lower": -1.0, "upper": 1.0},
            },
            "formula": "v ~ 1 + x + y",
        },
        {
            "name": "a",
            "prior": {
                "Intercept": {"name": "Uniform", "lower": 0.5, "upper": 3.0},
                "x": {"name": "Uniform", "lower": -1.0, "upper": 1.0},
                "y": {"name": "Uniform", "lower": -1.0, "upper": 1.0},
            },
            "formula": "a ~ 1 + x + y",
        },
    ],
)

In [ ]:
hssm_reg_v_a_angle

In [ ]:
hssm_reg_v_a_angle.graph()

In [ ]:
infer_data_reg_v_a = hssm_reg_v_a_angle.sample(
    sampler="nuts_numpyro", chains=2, cores=1, draws=1000, tune=1000
)

In [ ]:
az.summary(infer_data_reg_v_a, var_names = ['~a', '~v']) # , var_names=["~rt,response_a"])

In [ ]:
az.plot_trace(hssm_reg_v_a_angle.traces, 
              var_names = ['~v', '~a'],
              lines = [(key_, {}, param_dict_reg_v_a[key_]) for key_ in param_dict_reg_v_a])
plt.tight_layout()

We successfully recover our regression betas for `a`! Moreover, no warning signs concerning our chains.

### Case 4: Categorical covariates

In [ ]:
# Set up trial by trial parameters
x = np.random.choice(4, size=1000).astype(int)
x_offset = np.array([0, 1, -0.5, 0.75])
y

y = np.random.uniform(-1, 1, size=1000)
v_y = 0.3
v_reg_v = 0 + (v_y * y) + x_offset[x]

# rest
a_reg_v = 1.5
z_reg_v = 0.5
t_reg_v = 0.1

# base dataset
dataset_reg_v_cat = hssm.simulate_data(model = 'ddm',
                                   theta = dict(v = v_reg_v,
                                                a = a_reg_v,
                                                z = z_reg_v,
                                                t = t_reg_v),
                                   size = 1)

# Adding covariates into the datsaframe
dataset_reg_v_cat['x'] = x
dataset_reg_v_cat['y'] = y

In [ ]:
model_reg_v_cat = hssm.HSSM(
    data=dataset_reg_v_cat,
    model="angle",
    include=[
        {
            "name": "v",
            "formula": "v ~ 0 + C(x) + y",
            "link": "identity",
        }
    ],
)

In [ ]:
model_reg_v_cat.graph()

In [ ]:
infer_data_reg_v_cat = model_reg_v_cat.sample(
        sampler="nuts_numpyro", 
        chains=2, cores=1, 
        draws=1000, tune=500
        )

In [ ]:
az.plot_forest(infer_data_reg_v_cat, var_names = ["~v"])

## Hierarchical Inference

Let's try to fit a hierarchical model now. We will simulate a dataset with $15$ participants, with $200$ observations / trials for each participant. 

We define a group mean `mean_v` and a group standard deviation `sd_v` for the intercept parameter of the regression on `v`, which we sample from a corresponding normal distribution for each participant.

### Simulate Data

In [ ]:
# Make some hierarchical data

n_participants = 15  # number of participants
n_trials = 200  # number of trials per participant

sd_v = 0.5  # sd for v-intercept
mean_v = 0.5  # mean for v-intercept

data_list = []
for i in range(n_participants):
    # Make parameters for participant i
    v_intercept_hier = np.random.normal(mean_v, sd_v, size=1)
    x = np.random.uniform(-1, 1, size=n_trials)
    v_x_hier = 0.8
    y = np.random.uniform(-1, 1, size=n_trials)
    v_y_hier = 0.3
    v_hier = v_intercept_hier + (v_x_hier * x) + (v_y_hier * y)

    a_hier = 1.5
    t_hier = 0.5
    z_hier = 0.5

    # true_values = np.column_stack(
    #     [v, np.repeat([[1.5, 0.5, 0.5, 0.0]], axis=0, repeats=n_trials)]
    # )

    data_tmp = hssm.simulate_data(model = "ddm", 
                                  theta = dict(v = v_hier,
                                                     a = a_hier,
                                                     z = z_hier,
                                                     t = t_hier),
                                  size = 1)
    data_tmp['participant_id'] = i
    data_tmp['x'] = x
    data_tmp['y'] = y

    data_list.append(data_tmp)
    
# Make single dataframe out of participant-wise datasets
dataset_reg_v_hier = pd.concat(data_list)
dataset_reg_v_hier

We can now define our `HSSM` model. 

We specify the regression as `v ~ 1 + (1|participant_id) + x + y`. 

`(1|participant_id)` tells the model to create a *participant-wise* offset for the intercept parameter. The rest of the regression $\beta$'s is fit globally. 

As an **R** user you may recognize this syntax from the [lmer](https://www.rdocumentation.org/packages/lme4/versions/1.1-33/topics/lmer) package. 

Our [Bambi](https://bambinos.github.io/bambi/) backend is essentially a Bayesian version of [lmer](https://www.rdocumentation.org/packages/lme4/versions/1.1-33/topics/lmer), quite like the [BRMS](https://cran.r-project.org/web/packages/brms/index.html) package in **R**, which operates on top of [STAN](https://mc-stan.org/). 

As a previous [HDDM](https://hddm.readthedocs.io/en/latest/) user, you may recognize that now proper mixed-effect models are viable! 

You should be able to handle between and within participant effects naturally now!


### Basic Hierarchical Model

In [ ]:
model_reg_v_angle_hier = hssm.HSSM(
    data=dataset_reg_v_hier,
    model="angle",
    noncentered=True,
    include=[
        {
            "name": "v",
            "prior": {
                "Intercept": {
                    "name": "Normal",
                    "mu": 0.,
                    "sigma": 0.5,
                    "initval": 0.0,
                },
                "x": {"name": "Normal", "mu": 0.0, "sigma": 0.5, "initval": 0.0},
                "y": {"name": "Normal", "mu": 0.0, "sigma": 0.5, "initval": 0.0},
            },
            "formula": "v ~ 1 + (1|participant_id) + x + y",
            "link": "identity",
        }
    ],
)

In [ ]:
model_reg_v_angle_hier.graph()

In [ ]:
jax.config.update("jax_enable_x64", False)
model_reg_v_angle_hier.sample(
    sampler="nuts_numpyro", chains=2, cores=1, draws=1000, tune=1000
)

Let's look at the posteriors!

In [ ]:
az.plot_forest(model_reg_v_angle_hier.traces, var_names = ['~v', '~a'], combined=False)

## Model Comparison

Fitting single models is all well and good. We are however, often interested in comparing how well a few different models account for the same data. 

Through [ArviZ](https://python.arviz.org/en/stable/index.html), we have out of the box access to modern Bayesian Model Comparison. We will keep it simple here, just to illustrate the basic idea. 

### Scenario

The following scenario is explored. 

First we generate data from a `ddm` model with fixed parameters, specifically we set the `a` parameter to $1.5$. 

We then define two `HSSM` models:

1. A model which allows fitting all but the `a` parameter, which is fixed to $1.0$ (wrong)
2. A model which allows fitting all but the `a` parameter, which is fixed to $1.5$ (correct)

We then use the [ArviZ](https://python.arviz.org/en/stable/index.html)'s `compare()` function, to perform model comparison via `elpd_loo`. 

### Data Simulation

In [ ]:
# Parameters
param_dict_mod_comp = dict(v = 0.5, 
                           a = 1.5,
                           z = 0.5,
                           t = 0.2)

# Simulation
dataset_model_comp = hssm.simulate_data(model = "ddm",
                                        theta = param_dict_mod_comp,
                                        size = 500)

print(dataset_model_comp)

### Defining the Models

In [ ]:
# 'wrong' model
model_model_comp_1 = hssm.HSSM(
    data=dataset_model_comp,
    model="angle",
    a=1.0,
)

In [ ]:
# 'correct' model
model_model_comp_2 = hssm.HSSM(
    data=dataset_model_comp,
    model="angle",
    a=1.5,
)

In [ ]:
# 'wrong' model ddm
model_model_comp_3 = hssm.HSSM(
    data=dataset_model_comp,
    model="ddm",
    a=1.0,
)

In [ ]:
infer_data_model_comp_1 = model_model_comp_1.sample(
    sampler="nuts_numpyro",
    cores=1,
    chains=2,
    draws=1000,
    tune=1000,
    idata_kwargs=dict(
        log_likelihood=True
    ),  # model comparison metrics usually need this!
)

In [ ]:
infer_data_model_comp_2 = model_model_comp_2.sample(
    sampler="nuts_numpyro",
    cores=1,
    chains=2,
    draws=1000,
    tune=1000,
    idata_kwargs=dict(
        log_likelihood=True
    ),  # model comparison metrics usually need this!
)

In [ ]:
infer_data_model_comp_3 = model_model_comp_3.sample(
    sampler="nuts_numpyro",
    cores=1,
    chains=2,
    draws=1000,
    tune=1000,
    idata_kwargs=dict(
        log_likelihood=True
    ),  # model comparison metrics usually need this!
)

### Compare

In [ ]:
compare_data = az.compare(
    {
        "a_fixed_1(wrong)": model_model_comp_1.traces,
        "a_fixed_1.5(correct)": model_model_comp_2.traces,
        "a_fixed_1_ddm(wrong)": model_model_comp_3.traces,
    }
)

compare_data

Notice how the posterior weight on the `correct` model is close to (or equal to ) $1$ here. 
In other words model comparison points us to the correct model with 
a very high degree of certainty here!


We can also use the `.plot_compare()` function to illustrate the model comparison visually.

In [ ]:
az.plot_compare(compare_data)

Using the forest plot we can take a look at what goes wrong for the "wrong" model. 

To make up for the mistplaced setting of the `a` parameter, the posterior seems to compensate by
mis-estimating the other parameters.

In [ ]:
az.plot_forest([model_model_comp_1.traces, 
                model_model_comp_2.traces, 
                model_model_comp_3.traces], model_names = ["a_fixed_1(wrong)", 
                                                           "a_fixed_1.5(correct)",
                                                           "a_fixed_1(wrong)_ddm"])

## Closer look!

<center> <img src="images/pytensor_jax.png" height="400" width="400"> </center>

We have seen a few examples of HSSM models at this point. Add a model via a string, maybe toy a bit with with the priors and set regression functions for a given parameter. Turn it hierarchical...  Here we begin to peak a bit under the hood. 

After all, we want to encourage you to contribute models to the package yourself. 

Let's remind ourself of the `model_config` dictionaries that define model properties for us. Again let's start with the DDM.

In [ ]:
hssm.config.default_model_config["ddm"].keys()

The dictionary has a few high level keys. 

1. `response`

2. `list_params`

3. `description`

4. `likelihoods`


Let us take a look at the available `likelihoods`:

In [ ]:
hssm.config.default_model_config["ddm"]["likelihoods"]

For the DDM we have available all three types of likelihoods that HSSM deals with:

1. `analytical`
2. `approx_differentiable`
3. `blackbox`

Let's expand the dictionary contents more:

In [ ]:
hssm.config.default_model_config["ddm"]["likelihoods"]["analytical"]

We see three properties (key) in this dictionary, of which two are essential:

- The `loglik` field, which points to the likelihood function
- The `backend` field, which can be either `None` (defaulting to pytensor for `analytical` likelihoods), `jax` or `pytensor`
- The `bounds` field, which specifies bounds on a subset of the model parameters
- The `default_priors` field, which specifies parameter wise priors

If you provide `bounds` for a parameter, but no `default_priors`, a *Uniform* prior that respects the specified bounds will be applied. 

Next, let's look at the `approx_differentiable` part. 
The likelihood in this part is based on a [LAN]() which was available in [HDDM]() through the [LAN extension](). 

In [ ]:
hssm.config.default_model_config["ddm"]["likelihoods"]["approx_differentiable"]

We see that the `loglik` field is now a string that points to a `.onnx` file. 
[Onnx](https://onnx.ai/) is a meta framework for Neural Network specification, that allows translation between deep learning Frameworks. This is the preferred format for the neural networks we store in our model reservoir on [HuggingFace](https://huggingface.co/).

Moreover notice that we now have a `backend` field. We allow for two primary backends in the `approx_differentiable` field. 

1. `pytensor`
2. `jax` 

The `jax` backend assumes that your likelihood is described as a jax function, the `pytensor` backend assumes that your likelihood is described as a `pytensor` function. Ok not that surprising...

We won't dwell on this here, however the key idea is to provide users with a large degree of flexibility in describing their likelihood functions and moreover to allow targeted optimization towards MCMC sampler types that [PyMC]() allows us to access.

You can find a [dedicated tutorial](https://lnccbrown.github.io/HSSM/tutorial_likelihoods/#3-kinds-of-likelihoods) in the documentation, which describes the different likelihoods in much more detail. 

Instead, let's take a quick look at how these newfound insights can be used for custom model definition. 

In [ ]:
hssm_alternative_model = hssm.HSSM(
    data=dataset,
    model="ddm",
    loglik_kind="approx_differentiable",
)

In [ ]:
hssm_alternative_model.loglik_kind

In this case we actually built the model class with an `approx_differentiable` LAN likelihood, instead of the default `analytical` likelihood we used in the beginning of the tutorial. The assumed generative model remains the `ddm` however!

In [ ]:
hssm_alternative_model.sample(
    sampler="nuts_numpyro",
    cores=1,
    chains=2,
    draws=1000,
    tune=1000,
    idata_kwargs=dict(
        log_likelihood=False
    ),  # model comparison metrics usually need this!
)

In [ ]:
az.plot_forest(hssm_alternative_model.traces)

We can take this further and specify a completely custom likelihood. See the [dedicated tutorial](https://lnccbrown.github.io/HSSM/tutorial_likelihoods/#using-custom-likelihoods) for more examples!

We will see one specific example below to illustrate another type of likelihood function we have available for model building in HSSM, the *Blackbox* likelihood. 

## 'Blackbox' Likelihoods

<center> <img src="images/blackbox.png" height="400" width="400"> </center>

### What is a **Blackbox Likelihood Function**?

A *Blackbox Likelihood Function* is essentially any Python `callable` (function) that provides trial by trial likelihoods for your model of interest. What kind of computations are performed in this Python function is completely arbitrary. 

E.g. you could built a function that performs forward simulation from you model, constructs are kernel-density estimate for the resulting likelihood functions and evaluates your datapoints on this ad-hoc generated approximate likelihood.

What I just described is a once state-of-the-art method of performing simulation based inference on Sequential Sampling models, a precursor to LANs if you will.

We will do something simpler to keep it short and sweet, but really... the possibilities are endless!

<br> </br>
<center><img src="https://media.giphy.com/media/juq2OXONntuHfLc4Tn/giphy.gif" style="margin:auto" height="300" width="300"/></center>
<br> </br>


### Simulating simple dataset from the DDM

As always, let's begin by generating some simple dataset. 

In [ ]:
# Set parameters
param_dict_blackbox = dict(v = 0.5,
                           a = 1.5,
                           z = 0.5,
                           t = 0.5)

# Simulate
dataset_blackbox = hssm.simulate_data(model="ddm",
                                      theta=param_dict_blackbox,
                                      size = 1000)

### Define the likelihood

Now the fun part... we simply define a Python function `my_blackbox_loglik` which takes in our `data` as well as a bunch of model parameters (in our case the familiar `v`,`a`, `z`, `t` from the DDM).

The function then does some arbitrary computation inside (in our case e.g. we pass the data and parameters to the DDM log-likelihood from our predecessor package HDDM).

The important part is that inside `my_blackbox_loglik` anything can happen. We happen to call a little custom function that defines the likelihood of a DDM.

**Fun fact:**
It is de-facto the likelihood which is called by [HDDM](https://hddm.readthedocs.io/en/latest/).

In [ ]:
def my_blackbox_loglik(data, v, a, z, t, err=1e-8):
    data = data[:, 0] * data[:, 1]
    # Our function expects inputs as float64, but they are not guaranteed to
    # come in as such --> we type convert
    return hddm_wfpt.wfpt.pdf_array(
        np.float64(data),
        np.float64(v),
        0,
        np.float64(2 * a),
        np.float64(z),
        0,
        np.float64(t),
        0,
        err,
        1,
    )

### Define HSSM class with our Blackbox Likelihood

We can now define our HSSM model class as usual, however passing our `my_blackbox_loglik()` function to the `loglik` argument, and passing as `loglik_kind = blackbox`. 

The rest of the model config is as usual. Here we can reuse our `ddm` model config, and simply specify bounds on the parameters (e.g. your Blackbox Likelihood might be trustworthy only on a restricted parameters space). 

In [ ]:
blackbox_model = hssm.HSSM(
    data=dataset_blackbox,
    model="ddm",
    loglik=my_blackbox_loglik,
    loglik_kind="blackbox",
    model_config={
        "bounds": {
            "v": (-10.0, 10.0),
            "a": (0.5, 5.0),
            "z": (0.0, 1.0),
        }
    },
    t=bmb.Prior("Uniform", lower=0.0, upper=2.0, initval=0.1),
)

In [ ]:
sample = blackbox_model.sample()

**NOTE**:

Since *Blackbox likelihood functions* are assumed to not be differentiable, our default sampler for such likelihood functions is a `Slice` sampler. HSSM allows you to choose any other suitable sampler from the PyMC package instead. A bunch of options are available for gradient-free samplers.

### Results

In [ ]:
az.summary(sample)

In [ ]:
az.plot_trace(sample, lines = [(key_, {}, param_dict_blackbox[key_]) for key_ in param_dict_blackbox])
plt.tight_layout()

## HSSM Random Variables in PyMC

We covered a lot of ground in this tutorial so far. You are now a sophisticated HSSM user. 

It is therefore time to reveal a secret. We can actuallly peel back one more layer...
<br> </br>
<center><img src="https://media.giphy.com/media/3o6EhKIVvdhTkvjT8s/giphy.gif" style="margin:auto" height="300" width="300"/></center>
<br> </br>

Instead of letting HSSM help you build the entire model, we can instead use HSSM to construct valid [PyMC](https://www.pymc.io/welcome.html) distributions and then proceed to build a custom PyMC model by ourselves...
<br> </br>
<center><img src="https://media.giphy.com/media/3pDxFfu5yZblPUVnWY/giphy.gif" style="margin:auto" height="300" width="300"/></center>
<br> </br>

We will illustrate the simplest example below. It sets a pattern that can be exploited for much more complicated modeling exercises, which importantly go far beyond what our basic HSSM class may facilitate for you! 

See the [dedicated tutorial](https://lnccbrown.github.io/HSSM/notebooks/pymc/) in the [documentation](https://lnccbrown.github.io/HSSM/) if you are interested.

Let's start by importing a few convenience functions:

In [ ]:
# DDM models (the Wiener First-Passage Time distribution)
from hssm.likelihoods import logp_ddm_sdv, DDM
from hssm.distribution_utils import make_distribution

### Simulate some data

In [ ]:
# Simulate
param_dict_pymc = dict(v = 0.5,
                       a = 1.5,
                       z = 0.5,
                       t = 0.5,
                       theta = 0.0)

dataset_pymc = hssm.simulate_data(model="ddm",
                                  theta=param_dict_pymc,
                                  size=1000)

### Build a custom PyMC Model

We can now use our custom random variable `DDM` directly in a PyMC model.

In [ ]:
import pymc as pm

with pm.Model() as ddm_pymc:
    v = pm.Uniform("v", lower=-10.0, upper=10.0)
    a = pm.HalfNormal("a", sigma=2.0)
    z = pm.Uniform("z", lower=0.01, upper=0.99)
    t = pm.Uniform("t", lower=0.0, upper=0.6, initval=0.1)

    ddm = DDM("DDM", v=v, a=a, z=z, t=t, observed=dataset_pymc[['rt', 'response']].values)

Let's check the model graph:

In [ ]:
pm.model_to_graphviz(model=ddm_pymc)

Looks remarkably close to our HSSM version!

We can use PyMC directly to sample and finally return to ArviZ for some plotting!

In [ ]:
with ddm_pymc:
    ddm_pymc_trace = pm.sample()

In [ ]:
az.plot_trace(ddm_pymc_trace, lines = [(key_, {}, param_dict_pymc[key_]) for key_ in param_dict_pymc])

plt.tight_layout()

In [ ]:
az.plot_forest(ddm_pymc_trace)

### Alternative Models with PyMC

With very little extra work, we can in fact load any of the models accessible via HSSM. Here is an example, where we load the `angle` model instead. 

We first construction the likelihood function, using `make_likelihood_callable()`. 

Then we produce a valid `pymc.distribution` using the 
`make_distribution()` utility function.

Just like the `DDM` class above, we can then use this distribution inside a **PyMC** model.

In [ ]:
from hssm.distribution_utils import make_likelihood_callable, make_distribution

angle_loglik = make_likelihood_callable(loglik='angle.onnx', 
                                        loglik_kind='approx_differentiable', 
                                        backend='jax', 
                                        params_is_reg=[0, 0, 0, 0, 0])

ANGLE = make_distribution('angle', loglik = angle_loglik, list_params = hssm.defaults.default_model_config['angle']['list_params'])

Note that we need to supply the `params_is_reg` argument ("reg" for "regression"). 
This is a boolean vector, which specifies for each input to the likelihood function, whether or not it is defined to be "trial-wise", as is expected if the parameter
is the output e.g. of a regression function.

In [ ]:
import pymc as pm

# Angle pymc
with pm.Model() as angle_pymc:
    # Define parameters
    v = pm.Uniform("v", lower=-10.0, upper=10.0)
    a = pm.Uniform("a", lower=0.5, upper=2.5)
    z = pm.Uniform("z", lower=0.01, upper=0.99)
    t = pm.Uniform("t", lower=0.0, upper=0.6, initval=0.1)
    theta = pm.Uniform("theta", lower=-0.1, upper=1.0, initval= 0.2)

    # Our RV
    angle = ANGLE("DDM", v=v, a=a, z=z, t=t, theta=theta, observed=dataset_pymc[['rt', 'response']].values)

In [ ]:
with angle_pymc:
    idata_object = pm.sample(nuts_sampler="numpyro")

In [ ]:
az.plot_trace(idata_object, 
              lines = [(key_, {}, param_dict_pymc[key_]) for key_ in param_dict_pymc])

plt.tight_layout()

### Regression via PyMC

Finally to illustrate the usage of PyMC a little more elaborately, let us build a PyMC model with regression components.

In [ ]:
from typing import Optional
def make_params_is_reg_vec(reg_parameters: Optional[list] = None, 
                           parameter_names: Optional[list] = None):
    if (not isinstance(reg_parameters, list)) or (not isinstance(parameter_names, list)):
        raise ValueError('Both reg_parameters and parameter_names should be lists')

    bool_list = [0]*len(parameter_names)
    for param in reg_parameters:
        bool_list[parameter_names.index(param)] = 1
    return bool_list

In [ ]:
# Set up trial by trial parameters
v_intercept_pymc_reg = 0.3
x_pymc_reg = np.random.uniform(-1, 1, size=1000)
v_x_pymc_reg = 0.8
y_pymc_reg = np.random.uniform(-1, 1, size=1000)
v_y_pymc_reg = 0.3
v_pymc_reg = v_intercept + (v_x * x) + (v_y * y)

param_dict_pymc_reg = dict(v_Intercept = v_intercept_pymc_reg,
                           v_x = v_x_pymc_reg,
                           v_y = v_y_pymc_reg,
                           v = v_pymc_reg,
                           a = 1.5,
                           z = 0.5,
                           t = 0.1,
                           theta = 0.0)
                           
# base dataset
pymc_reg_data = hssm.simulate_data(model = 'ddm',
                                   theta = param_dict_pymc_reg,
                                   size = 1)

# Adding covariates into the datsaframe
pymc_reg_data['x'] = x
pymc_reg_data['y'] = y

# Make the boolean vector for params_is_reg argument
bool_param_reg = make_params_is_reg_vec(reg_parameters = ['v'], 
                                        parameter_names = hssm.defaults.default_model_config['angle']['list_params'])

angle_loglik = make_likelihood_callable(loglik='angle.onnx',
                                        loglik_kind='approx_differentiable', 
                                        backend='jax', 
                                        params_is_reg=bool_param_reg)

ANGLE = make_distribution('angle', 
                          loglik = angle_loglik, 
                          list_params = hssm.defaults.default_model_config['angle']['list_params'])

In [ ]:
import pytensor.tensor as pt
with pm.Model(coords = {'idx': pymc_reg_data.index,
                        'resp': ['rt', 'response'],
                        'features': ['x', 'y']}) as pymc_model_reg:

    # Features
    x_ = pm.Data('x', pymc_reg_data['x'].values, dims='idx')
    y_ = pm.Data('y', pymc_reg_data['y'].values, dims='idx')
    # Target
    obs = pm.Data('obs', pymc_reg_data[['rt', 'response']].values, dims=('idx', 'resp'))
    
    # Priors
    a = pm.Uniform("a", lower=0.5, upper=2.5)
    z = pm.Uniform("z", lower=0.01, upper=0.99)
    t = pm.Uniform("t", lower=0.0, upper=0.6, initval=0.1)
    theta = pm.Uniform("theta", lower=-0.1, upper=1.0, initval= 0.2)
    v_Intercept = pm.Uniform("v_Intercept", lower=-3, upper=3)
    v_betas = pm.Normal("v_beta", mu = [0, 0], sigma= 0.5, dims=('features'))
    
    # Regression equation
    v = pm.Deterministic('v', v_Intercept + pt.stack([x_, y_], axis=1) @ v_betas, dims='idx')
    
    # Our RV
    angle = ANGLE("angle", v=v.squeeze(), a=a, z=z, t=t, theta=theta, observed=obs, dims=('idx', 'resp'))

In [ ]:
with pymc_model_reg:
    idata_pymc_reg = pm.sample(nuts_sampler="numpyro", idata_kwargs={'log_likelihood': True})

In [ ]:
az.plot_forest(idata_pymc_reg, var_names=['~v'])

All layers peeled back, the only limit in your modeling endeavors becomes the limit of the PyMC universe!

<br> </br>

<center><img src="https://media.giphy.com/media/4ydWTcMBjimLbT1CHi/giphy.gif" style="margin:auto" height="300" width="300"/></center>


<center> Enjoy the exploration! </center>

# End